In [505]:
import numpy as np

def get_op_txt(digits, operator):
    outshape = digits.shape[:-1]
    digits_txt = digits.astype(str).reshape(-1, 2)
    op_txt = np.char.add(np.char.add(digits_txt[:, 0], operator), digits_txt[:, 1])
    
    return op_txt.astype(f'<U{np.max(np.char.str_len(op_txt))}').reshape(outshape)

def add(poss):
    ix = np.array([[i, j] 
                   for i in range(poss.shape[1]-1) 
                   for j in range(i+1, poss.shape[1])])
    label = get_op_txt(poss[:, ix], '+')
    result = np.sum(poss[:, ix], axis=2)
        
    return label, result, ix[:, 0], ix[:, 1]

def sub(poss):
    ix = np.array([[i, j] 
                   for i in range(poss.shape[1]) 
                   for j in range(poss.shape[1])
                   if i != j])
    label = get_op_txt(poss[:, ix], '-')
    result = np.sum(poss[:, ix] * [[[1, -1]]], axis=2)
    result = np.where(result < 0, -1, result)

    return label, result, ix[:, 0], ix[:, 1]

def mult(poss):
    ix = np.array([[i, j] 
                   for i in range(poss.shape[1]-1) 
                   for j in range(i+1, poss.shape[1])])
    label = get_op_txt(poss[:, ix], '*')
    result = np.prod(poss[:, ix], axis=2)

    return label, result, ix[:, 0], ix[:, 1]

def div(poss):
    ix = np.array([[i, j] 
                   for i in range(poss.shape[1]) 
                   for j in range(poss.shape[1])
                   if i != j])
    label = get_op_txt(poss[:, ix], '/')
    result = np.where(poss[:, ix][:, :, 1] == 0, 
                      np.maximum(poss[:, ix][:, :, 0], 1), 
                      poss[:, ix][:, :, 0])\
             / np.where(poss[:, ix][:, :, 1] == 0, 
                        np.minimum(-poss[:, ix][:, :, 0], -1), 
                        poss[:, ix][:, :, 1])    
    result = np.where(np.mod(result, 1) != 0, -1, result).astype(int)

    return label, result, ix[:, 0], ix[:, 1]

def moves_txt(moves):
    if moves.shape[1] == 1:
        return moves[:, 0]
    else:
        return np.char.add(np.char.add(moves[:, 0], '; '), moves_txt(moves[:, 1:]))

def all_targets(digits):
    # Initialize arrays, results dict
    poss = np.array([digits])
    moves = np.empty((1, 0), dtype=str)
    calc_to_target = {i: str(i) for i in digits}
    
    while poss.shape[1] > 1:
        # Do all possible operations, add to poss and moves arrays
        label, result, ix1, ix2 = (np.concatenate(i, axis=-1)
                                   for i in zip(*[f(poss) for f in [add, sub, mult, div]]))
        
        poss = np.repeat(poss, label.shape[1], axis=0)
        moves = np.repeat(moves, label.shape[1], axis=0)
        
        poss[np.arange(poss.shape[0]), np.tile(ix1, label.shape[0])] = result.reshape(-1)
        poss[np.arange(poss.shape[0]), np.tile(ix2, label.shape[0])] = poss[:, -1]
        poss = np.sort(poss[:, :-1], axis=1)
        
        moves = np.column_stack((moves, label.reshape(-1)))
        
        # Remove negatives, non-integers, duplicate outcomes
        sortix = np.lexsort([poss[:, i] for i in range(poss.shape[1]-1, -1, -1)])
        filt1 = np.any(poss[sortix] != np.append(-np.ones_like(poss[:1]), poss[sortix][:-1], axis=0), axis=1)
        filt2 = result.reshape(-1)[sortix] != -1
        
        poss = poss[sortix][filt1 & filt2]
        moves = moves[sortix][filt1 & filt2]
        
        # Add newly-reached numbers to calc_to_target dict
        newnum = result.reshape(-1)[sortix][filt1 & filt2]
        sortix2 = np.argsort(newnum)
        nnix = np.arange(poss.shape[0])\
                  [(newnum[sortix2] != np.append([-1], newnum[sortix2][:-1]))
                      [np.argsort(sortix2)]]
        
        calc_to_target = {**dict(zip(newnum[nnix].tolist(), 
                                     moves_txt(moves[nnix]).tolist())), 
                          **calc_to_target}

    return calc_to_target

def solve(digits, target):
    soln = all_targets(digits).get(target, 'No Solution')
    if soln == 'No Solution':
        print('No Solution')
    else:
        for i in soln.split(';'):
            print(f'{i.strip()} = {eval(i)}')

def impossible_targets(digits, n=10):
    solutions = all_targets(digits)
    impossible = list()
    target = 1
    while n > 0:
        if target not in solutions:
            impossible += [target]
            n -= 1
        target += 1
    
    return impossible

In [503]:
digits = [3,6,7,9,11,17]
target = 482
solve(digits, target)

7*11 = 77
6*77 = 462
17+462 = 479
3+479 = 482


In [502]:
impossible_targets(digits)

[1090, 1178, 1262, 1346, 1409, 1415, 1424, 1427, 1436, 1544]